In [ ]:
import math
import traceback
from flask import Flask
from flask import request
import numpy as np
from fractions import Fraction


def do_process(m):
    m.astype(int)
    m_size = len(m)
    row_sum = [sum(r) for r in m]
    print(row_sum)
    if row_sum[0] == 0:
        return [1, 1]
#     k = list(i for i in range(len(m))
    dic_all_row_zero = dict(zip(list(i for i in range(len(m))),list(0 if i == 0 else 1 for i in row_sum)))
    
    dic_all_row_zero = {k: v for k, v in sorted(dic_all_row_zero.items(), key=lambda item: item[1])}
    
    n_rows_all_zeros = list(dic_all_row_zero.values()).count(0)
    
    print("n_rows_all_zeros : ",n_rows_all_zeros)
    
    n_rows_all_non_zeros = len(m) - n_rows_all_zeros
    
    print("n_rows_all_non_zeros",n_rows_all_non_zeros)
    arr = np.array(m)
    
#     print(arr)
    
    arr_r = arr[list(dic_all_row_zero.keys()), :]
#     print(arr_r)
    arr_c = arr_r[:, list(dic_all_row_zero.keys())]
#     print(arr_c)
    new_r_sum = [sum(r) for r in arr_c]
    
    new_m = []
    for i in range(len(arr_c)):
        new_m.append(list(map(lambda x: 0 if new_r_sum[i] == 0 
                            else x / new_r_sum[i], arr_c[i])))

    newarr = np.array(new_m)
#     print(newarr)
    
    Q = newarr[n_rows_all_zeros:, n_rows_all_zeros:]
    print("Q: ",Q)
    R = newarr[n_rows_all_zeros:, :n_rows_all_zeros]
    print("R: ",R)
    ImQ = np.subtract(np.identity(n_rows_all_non_zeros), Q)
    print("ImQ: ",ImQ)
    F = np.linalg.inv(ImQ)
    print("F: ",F)
    FR = np.matmul(F, R)
    FRf = []
    for i in range(n_rows_all_non_zeros):
        FRf.append([Fraction.from_float(x).limit_denominator(max_denominator = 2**32) for x in FR[i]])

    print(FRf)
    maxd = max(list(map(lambda x: x.denominator, FRf[0])))
    indv = list(map(lambda x: (x*maxd).numerator, FRf[0]))
    return list(indv + [maxd])

#     return bb**2


app = Flask(__name__)
@app.route('/test', methods=['GET', 'POST'])
def test():
    try:
        m = request.form["m"]
        return str(do_process(m))
#         return do_process(bb)
    except Exception as e:
        return str(traceback.format_exc())


app.run(host='0.0.0.0', port=3000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.10.138:3000/ (Press CTRL+C to quit)
172.16.10.138 - - [21/Feb/2023 16:54:31] "POST /test HTTP/1.1" 200 -
